In [ ]:
from importlib import reload

import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch, ArrowStyle, Arc
from mpl_toolkits.mplot3d import proj3d
from mpl_toolkits.mplot3d import axes3d    
import numpy as np
from scipy.spatial.transform import Rotation as R

from thesis_utils.plotting import set_plotting, save_figure

from gwviz.mplutils import add_arrow3d

set_plotting()

In [ ]:
phi = 0.7 * np.pi
theta = 0.2 * np.pi
psi = 0.4 * np.pi

In [ ]:
rotation_sky = R.from_euler("yz", [theta, phi])
rotation_psi = R.from_euler("z", [-psi])

In [ ]:
source_frame_labels = (r"$\mathbf{e}_1$", r"$\mathbf{e}_2$", r"$\hat{\mathbf{n}}$")
source_frame_labels_sky = (r"$\mathbf{e}_1^\textrm{s}$", r"$\mathbf{e}_2^\textrm{s}$")
detector_frame_labels = (r"$\hat{\mathbf{p}}$", r"$\hat{\mathbf{q}}$", r"$\hat{\mathbf{z}}$")

In [ ]:
n_angle = 100
theta_pos = 0.5
phi_pos = 0.3
psi_pos = 0.5
phi_pad = np.array([0.0, 0.0, -0.1])
theta_pad = np.array([0, 0, 0.1])
psi_pad = np.array([0, 0, 0.1])
label_pad = np.array([0, 0, 0.1])

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.view_init(vertical_axis="z", azim=45, elev=20)
ax.set_axis_off()

origin = np.zeros(3)
detector_frame_axes = np.eye(3)

source_frame_distance = 2.0
source_frame_origin_unit = rotation_sky.apply(np.array([0, 0, 1]))
source_frame_origin = source_frame_distance * source_frame_origin_unit
source_frame_axes = detector_frame_axes.copy()
source_frame_axes[[0, 2]] *= -1
rotated_source_frame_axes = rotation_sky.apply(rotation_psi.apply(source_frame_axes))
source_frame_axes_sky = rotation_sky.apply(source_frame_axes)

axes_arrows_kwargs = dict(
    arrowstyle=ArrowStyle.Fancy(head_length=2.0, head_width=2.0),
)

for vec, label in zip(detector_frame_axes, detector_frame_labels):
    add_arrow3d(ax, origin, vec, color="k", **axes_arrows_kwargs)
    ax.text(*vec + label_pad, label, ha="center", va="center")

add_arrow3d(ax, origin, source_frame_origin, color="grey", ls=":")

for vec, label in zip(rotated_source_frame_axes, source_frame_labels):
    vec = source_frame_origin + vec
    add_arrow3d(ax, source_frame_origin, vec, color="k", **axes_arrows_kwargs)
    ax.text(*vec + label_pad, label, ha="center", va="center")

for vec, label in zip(source_frame_axes_sky[:2], source_frame_labels_sky):
    vec = source_frame_origin + vec
    add_arrow3d(
        ax,
        source_frame_origin, vec,
        color="grey",
        **axes_arrows_kwargs,
    )
    ax.text(*vec + label_pad, label)

los_projection_xy = source_frame_origin.copy()
los_projection_xy[2] = 0.0

add_arrow3d(ax, origin, los_projection_xy, color="grey", ls=":")
add_arrow3d(ax, source_frame_origin, los_projection_xy, color="grey", ls=":")

phi_vec = np.linspace(0, phi, n_angle)
phi_radius = 0.5
phi_angle = np.array([
    phi_radius * np.cos(phi_vec),
    phi_radius * np.sin(phi_vec),
    np.zeros(len(phi_vec)),
])
ax.plot(*phi_angle, color="blue")

ax.text(
    *phi_angle[:, int(phi_pos * n_angle)] + phi_pad,
    r"$\phi$",
    verticalalignment="center",
    horizontalalignment="center",
)

theta_vec = np.linspace(0, theta, n_angle)
theta_radius = 0.5
theta_angle = np.array([
    theta_radius * np.sin(theta_vec),
    np.zeros(len(theta_vec)),
    theta_radius * np.cos(-theta_vec),
])

theta_angle_rot = R.from_euler("z", [phi,]).apply(theta_angle.T).T

ax.plot(*theta_angle_rot, color="turquoise")
ax.text(
    *theta_angle_rot[:, int(theta_pos * n_angle)] + theta_pad,
    r"$\theta$",
    verticalalignment="center",
    horizontalalignment="center",
)

psi_radius = 0.5
psi_vec = np.linspace(0, -psi, n_angle) + np.pi
psi_angle = np.array([
    psi_radius * np.cos(psi_vec),
    psi_radius * np.sin(psi_vec),
    np.zeros(len(psi_vec)),
])
psi_angle_rot = source_frame_origin[:, np.newaxis] + rotation_sky.apply(psi_angle.T).T
ax.plot(
    *psi_angle_rot, color="purple"
)
ax.text(
    *psi_angle_rot[:, int(psi_pos * n_angle)] + psi_pad,
    r"$\psi$",
    verticalalignment="center",
    horizontalalignment="center",
)

xlims = [-1, 1]
ylims = [0, 1.3]
zlims = [0, 2]

ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_zlim(zlims)

ax.set_box_aspect((np.ptp(xlims), np.ptp(ylims), np.ptp(zlims)), zoom=1.0)

# plt.show()
fig.tight_layout()
save_figure(fig, "detector_angles", pad_inches=0.0)

In [ ]:
from pypdf import PdfWriter, PdfReader

In [ ]:
with open("figures/detector_angles.pdf","rb") as f:
    orig = PdfReader(f)
    cropped = PdfWriter()

    right = 0
    left = 1
    top = 0
    bottom = 500

    image = orig.pages[0]
    print(image.mediabox.lower_right)

    new_height = int(0.85 * image.mediabox.height)
    new_width = int(0.9 * image.mediabox.width)

    image.mediabox.lower_right = (
        image.mediabox.right, image.mediabox.bottom + (image.mediabox.height - new_height),
    )
    print(image.mediabox.lower_left)
    image.mediabox.lower_left = (
        image.mediabox.left + (image.mediabox.width - new_width), image.mediabox.bottom,
    )
    print(image.mediabox.lower_left)
    cropped.add_page(image)


with open("figures/detector_angles_cropped.pdf", "wb") as f:
    cropped.write(f)
